<font size="6">Reading SBML from uploaded files</font>

*This notebook is aimed at a niche bit of Computational Biology, but if you ignore all the bio bits it also explains how to do multiple user file uploads within a Jupyter notebook.*

Reading SBML via libSBML within a Jupyter notebook is easy enough from a local file on the host, but much harder for files uploaded by the user. As it took me a whole day to sort out a solution I thought I'd share it with the community.

The upload relies upon [peteut](https://github.com/peteut)'s IPython widget at https://github.com/peteut/ipython-file-upload. 

The widget is easy enough to set up but it took me a while to:

1) work out you have to write data to a global to access it from elsewhere in the notebook,

2) figure out how to do multiple file uploads (I needed to upload two source files to my notebook),

3) realise I needed to convert from 'unicode' to 'str' and use reader.readSBMLFromString() to create the document object.

# Packages

In [1]:
!pip install python-libsbml
!pip install ipywidgets
!pip install fileupload
!pip install uuid
!jupyter nbextension install --py fileupload
!jupyter nbextension enable --py fileupload

  Using cached fileupload-0.1.5-py2.py3-none-any.whl


  Using cached uuid-1.30.tar.gz
Failed to build uuid
  Running setup.py install for uuid ... done
Installing /Users/paul/miniconda2/lib/python2.7/site-packages/fileupload/static -> fileupload
Out of date: /usr/local/share/jupyter/nbextensions/fileupload/extension.js
Copying: /Users/paul/miniconda2/lib/python2.7/site-packages/fileupload/static/extension.js -> /usr/local/share/jupyter/nbextensions/fileupload/extension.js
Out of date: /usr/local/share/jupyter/nbextensions/fileupload/widget.js
Copying: /Users/paul/miniconda2/lib/python2.7/site-packages/fileupload/static/widget.js -> /usr/local/share/jupyter/nbextensions/fileupload/widget.js
Out of date: /usr/local/share/jupyter/nbextensions/fileupload/fileupload/widget.js
Copying: /Users/paul/miniconda2/lib/python2.7/site-packages/fileupload/static/fileupload/widget.js -> /usr/local/share/jupyter/nbextensions/fileupload/fileupload/widget.js
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or othe

In [2]:
from libsbml import *
from ipywidgets import widgets
import io, uuid
from IPython.display import display
import fileupload

# Load multiple source files

In [35]:
uploader = fileupload.FileUploadWidget()

def _cb(change):
    file_uuid = uuid.uuid4()
    decoded = io.StringIO(change['owner'].data.decode('utf-8'))
    filename = change['owner'].filename
    sbml = decoded.getvalue()
    sbml_strings[file_uuid] = sbml
    filenames[file_uuid] = filename
    print('Uploaded `{}` ({:.2f} kB)'.format(filename, len(decoded.read()) / 2 **10))

def _upload():
    _upload_widget = fileupload.FileUploadWidget()
    _upload_widget.observe(_cb, names='data')
    display(_upload_widget)

#The only way out of _cb is via a global, apparently.
#These globals hold the file contents and names
global sbml_strings
global filenames
sbml_strings = {}
filenames = {}
    
#Just change the number of instances here to upload different numbers of files
#This creates the upload buttons
numSources = 2
for num in range(0,numSources):
    print 'SBML file', num+1
    _upload()

SBML file 1


FileUploadWidget(label=u'Browse', _dom_classes=('widget_item', 'btn-group'))

SBML file 2


FileUploadWidget(label=u'Browse', _dom_classes=('widget_item', 'btn-group'))

Uploaded `Celegans_ecoli_inchi.xml` (4163.00 kB)
Uploaded `iCEL1273_inchi.xml` (3713.00 kB)


## Extract sources from global and compile list of libSBML documents

In [36]:
#Hold a list of SBML documents
listOfDocuments = []
#Keep track off source filenames
listOfFilenames = []
i = 1
for file_uuid in sbml_strings:
    print 'Source', i, 'is', filenames[file_uuid], '(', file_uuid, ')'
    sbml = str(sbml_strings[file_uuid])
    try:
        reader = SBMLReader()
        document = reader.readSBMLFromString(sbml)
        print 'Created document', i
        listOfDocuments.append(document)
        listOfFilenames.append(filenames[file_uuid])
    except:
        print 'Error reading model'
    i += 1

Source 1 is iCEL1273_inchi.xml ( bbc000a5-3ef4-4a67-9a12-0c2f21063678 )
Created document 1
Source 2 is Celegans_ecoli_inchi.xml ( fc430f97-846c-42b2-bbba-7bcd1feb215e )
Created document 2


# Show some model overview stats

In [34]:
j = 1
for doc in listOfDocuments:
    model = doc.getModel()
    print 'Document', j
    print 'Number of species:', model.getNumSpecies()
    print 'Number of reactions:', model.getNumReactions()    
    print 'Number of compartments:', model.getNumCompartments()
    print '\n'
    j += 1

Document 1
Number of species: 2357
Number of reactions: 1921
Number of compartments: 5


Document 2
Number of species: 1718
Number of reactions: 1985
Number of compartments: 3


